In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [6]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'wp244_rev_01_en_D8F5C63C-0B05-0150-00FADAD64B767E4E_44102.pdf'
paper_name = "Guidelines for identifying a controller or processor's lead supervisory authority"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [7]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 2, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

file = "../tmp/lead_sa.md"
with open(file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/lead_sa.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""
section_pattern = re.compile(r'(\d+)\.\s+(.*)')
subsection_pattern = re.compile(r'(\d\.\d+)\s+(.*)') 
point_pattern = re.compile(r'(\d\.\d\.\d+)\s+(.*)') 
annex_pattern = re.compile(r'^ANNEX - (.+)$') 
annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    section_match = section_pattern.match(entry)
    subsection_match = subsection_pattern.match(entry)
    point_match = point_pattern.match(entry)
    annex_match = annex_pattern.match(entry)

    if section_match:
        if annex_started:
            data.append(["", "", "", False, entry, "Annex"])
        else:
            match = section_match
            section = match.group(1)
            subsection = ""
            point = ""
            heading = True
            text = match.group(2)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
    elif subsection_match:
        match = subsection_match
        section = section
        subsection = match.group(1)
        point = ""
        heading = True
        text = match.group(2)
        section_reference = subsection
        data.append([section, subsection, point, heading, text, section_reference])
    elif point_match:
        match = point_match
        section = section
        subsection = subsection
        point = match.group(1)
        heading = True
        text = match.group(2)
        section_reference = point
        data.append([section, subsection, point, heading, text, section_reference])
    elif annex_match:
        match = annex_match
        annex_started = True
        section = "Annex"
        subsection = ""
        point = ""
        heading = True
        text = match.group(1)
        section_reference = section
        data.append([section, subsection, point, heading, text, section_reference])
    
    else:
        if annex_started:
            data.append(["", "", "", False, entry, "Annex"])
        else:
            section = section
            subsection = subsection
            point = point
            heading = False
            text = entry
            section_reference = section_reference
            
            data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[100:120]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
100,,,,False,b. In a case involving a controller and a proc...,Annex
101,,,,False,i. Check if the controller is established in t...,Annex
102,,,,False,ii. Identify the lead supervisory authority of...,Annex
103,,,,False,iii. The (non-lead) supervisory authority com...,Annex
104,,,,False,c. In a case involving only a processor:,Annex
105,,,,False,i. Identify the processor's place of central a...,Annex
106,,,,False,ii. If the processor has no central administra...,Annex
107,,,,False,establishment in the EU where the main process...,Annex
108,,,,False,d. In a case involving joint controllers:,Annex
109,,,,False,i. Check if the joint controllers are establis...,Annex


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 37 augmented with footnotes
Row 44 augmented with footnotes


In [6]:
file = "../../inputs/documents/lead_sa.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [10]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.lead_sa
importlib.reload(gdpr_rag.documents.lead_sa)
from gdpr_rag.documents.lead_sa import Lead_SA

path_to_manual_as_csv_file = "../../inputs/documents/lead_sa.parquet"

doc = Lead_SA(path_to_manual_as_csv_file)


In [16]:
from IPython.display import Markdown, display

section = "1"
section = "2.8"
section = "Annex"

section = "2.1.3"
section = "3.3"

section = "2.1"


print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


2 Steps to identify the lead supervisory authority
2.1 Identify the 'main establishment' for controllers


# 2 Steps to identify the lead supervisory authority

## 2.1 Identify the 'main establishment' for controllers

In order to establish where the main establishment is, it is firstly necessary to identify the central administration of the data controller in the EU, if any.[^1]  The approach implied in the GDPR is that the central administration in the EU is the place where decisions about the purposes and means of the processing of personal data are taken and this place has the power to have such decisions implemented.

The essence of the lead authority principle in the GDPR is that the supervision of cross-border processing should be led by only one supervisory authority in the EU. In  cases where decisions relating to different cross-border processing activities are taken within the EU central administration, there will be a single lead supervisory authority for the various data processing activities carried out by the multinational company. However, there may be cases where an establishment other than the place of central administration makes autonomous decisions concerning the purposes and means of a specific processing activity. This means that there can be situations where more than one lead authority can be identified, i.e. in cases where a multinational company decides to have separate decision making centres, in different countries, for different processing activities.

It is worth recalling, that where a multinational company centralises all the decisions relating to the purposes and means of processing activities in one of its establishments in the EU (and that establishment has the power to implement such decisions), only one lead supervisory authority will be identified for the multinational.

In these situations it will be essential for companies to identify precisely where the decisions on purpose and means of processing are taken. Correct identification of the main establishment is in the interests of controllers and processors because it provides clarity in terms of which supervisory authority they have to deal with in respect of their various compliance duties under the GDPR. These may include, where relevant, designating a data protection officer or consulting for a risky processing activity that the controller cannot mitigate by reasonable means. The relevant provisions of the GDPR are intended to make these compliance tasks manageable.   The examples below illustrate this:

Example 1:

A food retailer has its headquarters (i.e. its 'place of central administration') in Rotterdam, Netherlands. It has establishments in various other EU countries, which are in contact with individuals there. All establishments make use of the same software to process consumers' personal data for marketing purposes. All the decisions about the purposes and means of the processing of consumers' personal data for marketing purposes are taken within its Rotterdam headquarters. This means that the company's lead supervisory authority for this cross border processing activity is the Netherlands supervisory authority.

Example 2:

A bank has its corporate headquarters in Frankfurt, and all[^2] its banking processing activities are organised from there, but its insurance department is located in Vienna. If the establishment in Vienna has the power to decide on all insurance data processing activity  and to implement these decisions for the whole EU, then as foreseen in Art 4(16) of the GDPR, the Austrian supervisory authority would be the lead authority in respect of the cross border processing of personal data for  insurance purposes, and the German authorities (Hessen supervisory authority) would supervise the processing of personal data for banking purposes, wherever the clients are located.[^3]

  
[^1]: The GDPR is relevant for the EEA and will apply after its incorporation into the EEA Agreement. The GDPR is currently under scrutiny for incorporation, see http://www.efta.int/eea-lex/32016R0679  
[^2]: In the context of processing personal data for banking purposes, we recognise that are many different processing activities involved in this. However, to simplify matters, we address all of them as a single purpose. The same is true of processing done for insurance purposes.  
[^3]: It should be recalled also that the GDPR provides for the possibility of local oversight in specific cases. See  Recital (127): "Each supervisory authority not acting as the lead supervisory authority should be competent to handle local cases where the controller or processor is established in more than one Member State, but the subject matter of the specific processing concerns only processing carried out in a single Member State and involves only data subjects in that single Member State, for example, where the subject matter concerns the processing of employees' personal data in the specific employment context of a Member State." This principle means that the supervision of HR data connected to local employment context could fall to several supervisory authorities.